In [ ]:
from IPython.display import display, Math, Latex, HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''') 

In [ ]:
from influxdb import DataFrameClient
from influxdb import InfluxDBClient
import json
import pandas as pd

import datetime 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
#from datetime import datetime
import dateutil.parser
init_notebook_mode(connected=True)

In [ ]:
colors=['#F2F3F4', '#222222', '#F3C300', '#875692', '#F38400', '#A1CAF1', '#BE0032', '#C2B280', 
        '#848482', '#008856', '#E68FAC', '#0067A5', '#F99379', '#604E97', '#F6A600', '#B3446C',
        '#DCD300', '#882D17', '#8DB600', '#654522', '#E25822', '#2B3D26']

In [ ]:
def barplot_averages(df,column, title,index_col='SK_PI', margin=False, line='',xtitle="Device Numbers", ytitle="Mbps"):
    max_df = df.groupby(index_col)[column].max().reset_index()
    max_df.columns = [index_col, column+'_MAX']
    med_df = df.groupby(index_col)[column].median().reset_index()
    med_df.columns = [index_col, column+'_MEDIAN']
    avg_df = df.groupby(index_col)[column].mean().reset_index()
    avg_df.columns = [index_col, column+'_MEAN']
    med_max_avg_df=pd.merge(pd.merge(max_df, med_df,on=index_col),avg_df, on=index_col)
    trace1 = go.Bar(
                x=med_max_avg_df[index_col],
                y=med_max_avg_df[column+'_MEAN'],
                name='Avg',
        )
    trace2 = go.Bar(
                x=med_max_avg_df[index_col],
                y=med_max_avg_df[column+'_MAX'],
                name='Max',

        )
    trace3 = go.Bar(
                x=med_max_avg_df[index_col],
                y=med_max_avg_df[column+'_MEDIAN'],
                name='Median',

        )
    data = [trace1, trace2, trace3]
    if line:
        data.append(line)
    if margin:
        layout = go.Layout(
                title=title,
                xaxis=dict(title=xtitle),
                yaxis=dict(title=ytitle),
                margin = dict(
                l= 60,
                r= 30,
                t= 50,
                b= 200
                )
            )
    else:
        layout = go.Layout(title=title,xaxis=dict(title=xtitle),yaxis=dict(title=ytitle),)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [ ]:
with open('../credentials.json', 'r') as f_credentials:
    credentials_config = json.load(f_credentials)

In [ ]:
host=credentials_config['influxdb_host']
port=8086
dbname = 'net_speed_md'
client = InfluxDBClient(host, port, '', '', dbname)

In [ ]:
query_unique_devices = "SHOW TAG VALUES FROM PING WITH KEY=SK_PI;"
result_unique_devices = client.query(query_unique_devices)
points_unique_devices = result_unique_devices.get_points()
device_numbers=[]
for point in points_unique_devices:
    device_numbers.append(point['value'])
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
#print(device_numbers)

In [ ]:
query_ping_counts = 'SELECT COUNT(PING) FROM PING WHERE PING!=0 GROUP BY SK_PI;'
result_ping_counts = client.query(query_ping_counts)
query_ping_counts_dec = "SELECT COUNT(PING) FROM PING WHERE time >= now()-4w  AND PING!=0 GROUP BY SK_PI ;"
result_ping_counts_dec = client.query(query_ping_counts_dec)

In [ ]:
query_pingdroprate_counts = 'SELECT COUNT(PING_DROPRATE) FROM PING WHERE PING_DROPRATE!=0 GROUP BY SK_PI;'
result_pingdroprate_counts = client.query(query_pingdroprate_counts)
query_pingdroprate_counts_dec = 'SELECT COUNT(PING_DROPRATE) FROM PING WHERE time >= now()-4w AND PING_DROPRATE!=0 GROUP BY SK_PI;'
result_pingdroprate_counts_dec = client.query(query_pingdroprate_counts_dec)

In [ ]:
ping_counts=[]
pingdroprate_counts=[]
for device in device_numbers:
    points_ping_counts=result_ping_counts.get_points(tags={'SK_PI':str(device)})
    points_pingdroprate_counts=result_pingdroprate_counts.get_points(tags={'SK_PI':str(device)})
    point_ping=0
    point_pingdroprate=0
    for point in points_ping_counts:
        point_ping=point['count']
    count_ping=point_ping
    for point in points_pingdroprate_counts:
        point_pingdroprate=point['count']
    count_pingdroprate=point_pingdroprate
    ping_counts.append(count_ping+count_pingdroprate)
    pingdroprate_counts.append(count_pingdroprate)

In [ ]:
ping_counts_dec=[]
device_numbers_dec = []
pingdroprate_counts_dec = []
ping_sent_counts_dec = []
for device in device_numbers:
    points_ping_counts_dec=result_ping_counts_dec.get_points(tags={'SK_PI':str(device)})
    points_pingdroprate_counts_dec=result_pingdroprate_counts_dec.get_points(tags={'SK_PI':str(device)})
    point_ping=0
    point_pingdroprate=0
    for point in points_ping_counts_dec:
        point_ping=point['count']
    for point in points_pingdroprate_counts_dec:
        point_pingdroprate=point['count']
    if (point_pingdroprate+point_ping!=0): 
        device_numbers_dec.append(device)
    ping_counts_dec.append(point_pingdroprate+point_ping)
    pingdroprate_counts_dec.append(point_pingdroprate)
    ping_sent_counts_dec.append(point_ping)
#print("Devices, that have data in last 4 weeks: ",device_numbers_dec)

In [ ]:
client_df = DataFrameClient(host, port, '', '', dbname)
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf';"
result_upload= client_df.query(query_upload)
upload_df = result_upload['SPEEDTEST_UPLOAD']

In [ ]:
upload_df.reset_index(level=0, inplace=True)
upload_df['index']=upload_df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')
upload_df['index'] = pd.to_datetime(upload_df['index'])
upload_df['SK_PI']=pd.to_numeric(upload_df['SK_PI'])
upload_df.rename(columns={'index':'time'}, inplace=True)
upload_df = upload_df[upload_df.UPLOAD != 0]

In [ ]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
four_weeks_ago = datetime.date.today() - datetime.timedelta(days=28)
upload_df_last4weeks = upload_df[upload_df['time']>four_weeks_ago]
points_by_device_las4weeks = upload_df_last4weeks.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
merged_results=pd.merge(points_by_device, points_by_device_las4weeks, on='SK_PI', how='outer')
merged_results['counts_y']=merged_results['counts_y'].fillna(0)
merged_results['result']=merged_results['counts_x'].sub(merged_results['counts_y'], axis=0)

# ConnectIN draft analysis

This PDF contains some initial observations of Internet speed test data being collected from various locations in Manitoba as part of the ConnectIN project. Note that these are initial observations and are still in draft format. 

At the time of the analysis in early January 2019, data had been collected from a total of 18 devices, primarily located in Manitoba. Device locations were not known yet at this time with the exception of device number 3, which is a control device set up in Cybera's offices in Calgary. 



# 1. Statistics by device

This data was collected from speedtest.net, which reports on download, upload and ping (latency) of each device. Speedtests are run every 3 hours and 42 minutes. On average, the data seen here corresponds to about 30 days of data. 

In [ ]:
query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf';"
result_download= client_df.query(query_download)
download_df = result_download['SPEEDTEST_DOWNLOAD']

download_df.reset_index(level=0, inplace=True)
download_df['index']=download_df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')
download_df['index'] = pd.to_datetime(download_df['index'])
download_df['SK_PI']=pd.to_numeric(download_df['SK_PI'])

download_df.rename(columns={'index':'time'}, inplace=True)
download_df = download_df[download_df.DOWNLOAD != 0]

In [ ]:
query_ping = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER!='iperf';"
result_ping= client_df.query(query_ping)
ping_df = result_ping['SPEEDTEST_PING']

ping_df.reset_index(level=0, inplace=True)
ping_df['index']=ping_df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')
ping_df['index'] = pd.to_datetime(ping_df['index'])
ping_df['SK_PI']=pd.to_numeric(ping_df['SK_PI'])

ping_df.rename(columns={'index':'time'}, inplace=True)
ping_df = ping_df[ping_df.PING != 1800000.000] ## Removed outier that was found in the ms sql table

In [ ]:
query_ping_max_dec = "SELECT MAX(PING), MEAN(PING), MEDIAN(PING) FROM PING WHERE PING!=0  GROUP BY SK_PI;"
result_ping_max_dec = client.query(query_ping_max_dec)

In [ ]:
device_max_dec=[]
device_mean_dec = []
device_median_dec = []
for device in device_numbers:
    points_max_dec=result_ping_max_dec.get_points(tags={'SK_PI':str(device)})
    for point in points_max_dec:
        device_median_dec.append(point['median'])
        device_max_dec.append(point['max'])
        device_mean_dec.append(point['mean'])

In [ ]:
trace1 = go.Bar(
            x=device_numbers,
            y=device_mean_dec,
            name='Mean',
    )
trace2 = go.Bar(
            x=device_numbers,
            y=device_max_dec,
            name='Max',
    
    )
trace3 = go.Bar(
            x=device_numbers,
            y=device_median_dec,
            name='Median',
    
    )
data = [trace1, trace2, trace3]
layout = go.Layout(
       # barmode='stack',
        title="Collectd: Maximum, mean and median ping latency per device"
    )

#fig1 = go.Figure(data=data, layout=layout)
#iplot(fig1)

In [ ]:
upload_line=go.Scatter(x=device_numbers,y=[10] * 18, mode='markers',marker=dict(color='red'), name='10Mps')
download_line=go.Scatter(x=device_numbers,y=[50] * 18, mode='markers',marker=dict(color='red'), name='50Mps')
barplot_averages(download_df,'DOWNLOAD',"Download speed", line=download_line)
barplot_averages(upload_df,'UPLOAD',"Upload speed",line=upload_line)

In [ ]:
barplot_averages(ping_df,'PING',"Ping latency",ytitle="Miliseconds")

## Summary

The plot of the download speeds shows that only devices 8 and 14 have averages or medians above the CRTC target of 50 Mbps and for upload speeds, devices 8, 9, 11, 14, 15, 17 have averages or medians above 10 Mbps. This is somewhat surprising, given that upload speeds are usually much worse. Given that these devices are all set up in professional buildings (e.g. schools, offices, etc), it is a bad sign that only few devices meet the CRTC targets. At the same time, one important caveat to remember is that we do not know how much bandwidth each organization is purchasing. For example, we would not expect to see speeds of 50 Mbps if only 10 Mbps are being purchased. 


### Brief note on box plots

Boxplots are an excellent way to summarize statistical data as they readily display how the data is distributed - giving you a better idea as to the range, and more importantly, the distinguishability of your data. However if you've never seen one before, here's a few important things to note about boxplots and what everything they display means. To begin, here is a labeled boxplot

![alt-text](https://cdn-images-1.medium.com/max/800/1*2c21SkzJMf3frPXPAR_gZA.png)

What we see here is a few important pieces 
1. The 'box' itself is centered on the median of the data, and has width equal to the interquartile range, which basically says that the box contains 50% of the data.
2. The lines that extend out are known as "whiskers" and extend from the bottom of the box, to a distance equal to 1.5 times the width of the box on each side (1.5 times the inter quartile range)
3. Finally, any points outside of the whiskers are drawn individually 

In [ ]:
data1=[]
for device in device_numbers:
    trace=go.Box(
    y=download_df.loc[download_df['SK_PI']==device]['DOWNLOAD'], name=device, marker=dict(color=colors[device])
)
    data1.append(trace)
data1.append(go.Scatter(x=device_numbers,y=[50] * 18, mode='markers',marker=dict(color='red'), name='50Mps'))
layout1 = go.Layout(
            title="Download speed",
            xaxis=dict(title="Device Number"),
            yaxis=dict(title="Mbps"),
        )

fig1 = go.Figure(data=data1, layout=layout1)
iplot(fig1)

data2=[]
for device in device_numbers:
    trace=go.Box(
    y=upload_df.loc[upload_df['SK_PI']==device]['UPLOAD'], name=device, marker=dict(color=colors[device])
)
    data2.append(trace)
data2.append(go.Scatter(x=device_numbers,y=[10] * 18, mode='markers',marker=dict(color='red'), name='10Mps'))
layout2 = go.Layout(
            title="Upload speed",
            xaxis=dict(title="Device Number"),
            yaxis=dict(title="Mbps"),
        )

fig = go.Figure(data=data2, layout=layout2)
iplot(fig)

data3=[]
for device in device_numbers:
    trace=go.Box(
    y=ping_df.loc[ping_df['SK_PI']==device]['PING'], name=device,marker=dict(color=colors[device])
)
    data3.append(trace)

layout3 = go.Layout(
            title="Ping latency",
            xaxis=dict(title="Device Number"),
            yaxis=dict(title="Miliseconds"),
        )

fig = go.Figure(data=data3, layout=layout3)
iplot(fig)

##  Summary

Looking at the box plot, device 14 shows a lot of variation in upload and download speeds, whereas many of the other devices show narrowly distributed data. Device 5 and 16 show a large number of outliers in regards to latency. Interestingly, devices displaying high bandwidth speeds show narrowly distributed latency and devices with low speeds have larger variability in latency. 

# 2. Statistics by Internet Service Provider

Each time a test is run, speedtest.net records which service provider the test originates from. At this time we are unclear about the exact methodology used by speedtest for determining the provider. 

In [ ]:
providers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x["PROVIDER"].unique()).apply(pd.Series)
providers_per_device['provider_count'] = providers_per_device.apply(lambda x: x.count(), axis=1)
#providers_per_device.head()
provider_counts_per_device = pd.Series(providers_per_device['provider_count']).value_counts().reset_index()
#provider_counts_per_device
#data = [go.Pie(
#            labels=provider_counts_per_device['index'],
#            values=provider_counts_per_device["provider_count"],
#    )]
#layout = go.Layout(
#        #barmode='stack',
#        title="Number of providers per device"
#    )

#fig1 = go.Figure(data=data, layout=layout)
#iplot(fig1)
points_by_device_by_provider=upload_df.groupby(['SK_PI', 'PROVIDER']).size().reset_index()
providers=upload_df['PROVIDER'].unique()
data=[]
i=0
for provider in providers:
    prov=[]
    for device in device_numbers:
        by_provider=points_by_device_by_provider.loc[(points_by_device_by_provider['SK_PI']==device)&(points_by_device_by_provider['PROVIDER']==provider)]
        if not by_provider.empty:
            result=by_provider[0].iloc[0]/points_by_device.loc[points_by_device['SK_PI']==device]['counts'].iloc[0]*100
            prov.append(result)
        else:
            prov.append(0)
    trace = go.Bar(x=device_numbers,y=prov, name = provider, marker=dict(color=colors[i]))
    i=i+1
    data.append(trace)
    layout = go.Layout(
        barmode='stack',
        title="Percentage of service providers by device",
        xaxis=dict(title="Device Number"),
        yaxis=dict(title="Percentage of data"),
    )
fig2 = go.Figure(data=data, layout=layout)
iplot(fig2)

In [ ]:
upload_line=go.Scatter(x=providers,y=[10] * 18, mode='markers',marker=dict(color='red'), name='10Mps')
download_line=go.Scatter(x=providers,y=[50] * 18, mode='markers',marker=dict(color='red'), name='50Mps')
barplot_averages(upload_df,'UPLOAD',"Upload speed",index_col='PROVIDER',line=upload_line, xtitle="Provider",margin=True)
barplot_averages(download_df,'DOWNLOAD',"Download speed", index_col='PROVIDER',line=download_line,xtitle="Provider",margin=True)
barplot_averages(ping_df,'PING',"Ping Latency", index_col='PROVIDER',xtitle="Provider",ytitle="Miliseconds",margin=True)

In [ ]:
data3=[]
i=0
for provider in providers:
    i=i+1
    trace=go.Box(
    y=ping_df.loc[ping_df['PROVIDER']==provider]['PING'], name=provider,marker=dict(color=colors[i])
)
    data3.append(trace)

layout3 = go.Layout(
            title="Ping latency",
            xaxis=dict(title="Provider"),
            yaxis=dict(title="Miliseconds"),
            margin = dict(
                l= 60,
                r= 30,
                t= 50,
                b= 200
                )
        )

fig = go.Figure(data=data3, layout=layout3)
iplot(fig)

## Summary:
Data has been collected on 10 internet service providers. Most of the devices have one provider, devices 8 and 14 have used 2 providers. This could be because the device was moved from one location to another or alternatively, it might reflect that certain devices utilize primary and secondary internet service providers. 

Bell and Keewaytinook were used by multiple devices, while the rest of the providers were only used by one device.  
   
For upload speeds, the CRTC target is 10Mbps: every provider except Bell MTS, High Speed Crow and  TeraGo Networks, the average uploadload speed is above 10.
   
For download speeds, the CRTC target is 50Mbps. Only Commstream Communications, MERLIN and Manitoba Hydro Internationl reached this number.

The largest ping latencies are Bell MTS, TeraGo Networks and Keewaytinook Okimakanak.

Based on conversation with the steering committee, there is some uncertainty about the accuracy of the service providers recorded here. This is something that will require further investigation going forward. 

# 3. Number of data points, device reporting times

Below we show a summary of the number of datapoints collected and over which period of time devices reported data. Two methods for collecting data are shown below: collectd and speedtest. The collectd tests report lesss data, but tests run far more frequently every 5 seconds, while speedtest reports on more data (upload, download speeds, and latency), but tests are run only every 3 hours and 42 minutes. 

In [ ]:
trace1 = go.Bar(
            x=device_numbers,
            y=ping_counts_dec,
            name='Last 4 weeks', 
            marker=dict(color=colors[1])
    )
trace2 = go.Bar(
            x=device_numbers,
            y=[a - b for a, b in zip(ping_counts, ping_counts_dec)],
            name='The rest of the time',
            marker=dict(color=colors[2])
    )

trace3 = go.Bar(
            x=merged_results["SK_PI"],
            y=merged_results["counts_y"],
            name='Last 4 weeks',
            marker=dict(color=colors[1])
    )
trace4= go.Bar(
            x=merged_results["SK_PI"],
            y=merged_results['result'],
            name='The rest of the time',
            marker=dict(color=colors[2])
    )
#fig = tools.make_subplots(rows=1, cols=2)#, shared_xaxes=True)
#fig.append_trace(trace1, 1,1)
#fig.append_trace(trace2, 1,1)
#fig.append_trace(trace3, 1,2)
#fig.append_trace(trace4, 1,2)
#fig['layout'].update(barmode='stack',title="Number of datapoints: collectd data on the right, speedtest data on the left")#, width=1000)
data1 = [trace1, trace2]
layout1 = go.Layout(
        barmode='stack',
        title="Number of datapoints :collectd"
    )

fig1 = go.Figure(data=data1, layout=layout1)
iplot(fig1)
data2=[trace3, trace4]
layout2 = go.Layout(
        barmode='stack',
        title="Number of datapoints :speedtest"
    )

fig2 = go.Figure(data=data2, layout=layout2)
iplot(fig2)

In [ ]:
query_ping_last = "SELECT LAST(PING), time FROM PING GROUP BY SK_PI;"
result_ping_last = client.query(query_ping_last)

In [ ]:
query_ping_first = "SELECT FIRST(PING), time FROM PING  GROUP BY SK_PI;"
result_ping_first = client.query(query_ping_first)

In [ ]:
data=[]
for device in device_numbers:
    points_ping_last=result_ping_last.get_points(tags={'SK_PI':str(device)})
    points_ping_first=result_ping_first.get_points(tags={'SK_PI':str(device)})
    first=0
    last=0
    for point in points_ping_first:
        first=dateutil.parser.parse(point['time']).strftime('%Y-%m-%d %H:%M:%S')
    for point in points_ping_last:
        last=dateutil.parser.parse(point['time']).strftime('%Y-%m-%d %H:%M:%S')
   # print("Device: ", device,"  was reporting from ", first, " to ", last)
    trace = go.Scatter(x=[first,last],y=[device,device], name = str(device),marker=dict(color=colors[device]))
    data.append(trace)
layout = dict(title = "Device reporting times - collectd")
fig1 = go.Figure(data=data, layout=layout)
iplot(fig1)

start_times=upload_df.groupby('SK_PI')['time'].min()
end_times=upload_df.groupby('SK_PI')['time'].max()
device_numbers=sorted(upload_df['SK_PI'].unique())
#print(device_numbers)
data=[]
for device in device_numbers:
    time_last=end_times[device]
    time_first=start_times[device]
    #print("Device: ", device,"  was reporting from ", time_first, " to ", time_last)
    trace = go.Scatter(x=[time_first,time_last],y=[device,device], name = str(device),marker=dict(color=colors[device]))
    data.append(trace)
layout = dict(title = "Device reporting times   - speedtest")
fig2 = go.Figure(data=data, layout=layout)
iplot(fig2)

## Summary: 

There is at least eom data from 18 devices. Devices 1, 2, 4, 6, 8 started reporting speedtest data and then stopped. Device 13 has not sent any speedtest data (sent several collectd data points) and devices 10, 13, 14 are not sending collectd data but sending speedtest data.    

More detailed timeline can be found in [grafana](https://grafana-connectin.cybera.ca) (login required).

The devices that are no longer reporting data should be followed-up with and reset. It appears that the speedtest may be running more reliably, as connectd tests have shown a higher failure rate than speedtest tests. 